In [9]:
import pandas as pd
import numpy as np
import seaborn as sb

import statsmodels.api as sm
import datetime
from sklearn.metrics import r2_score
%matplotlib inline

#data = pd.read_pickle('player_stats_year_wise.pkl')
#data
ori_data = pd.read_pickle('tfa_working_ver_1')
ori_data.head()

,player_rank,player,pos,age,games,games_started,min_per_game,field_goals,field_goals_attempts,field_goals_pct,...,turnovers,personal_fouls,points_per_game,year,salary,new_pos,team_id,team_points,team,team_rank
0,65,aaron brooks,PG,33,32,1,5.9,0.9,2.2,0.406,...,0.3,0.9,2.3,2018,2116955.0,back,MIN,0.573,Minnesota Timberwolves,13
1,179,aaron gordon,PF,22,58,57,32.9,6.5,14.9,0.434,...,1.8,1.9,17.6,2018,5504420.0,front,ORL,0.305,Orlando Magic,26
2,363,abdel nader,SF,24,48,1,10.9,1.0,3.1,0.336,...,0.7,0.9,3.0,2018,1167333.0,front,BOS,0.671,Boston Celtics,4
3,227,al horford,C,31,72,72,31.6,5.1,10.5,0.489,...,1.8,1.9,12.9,2018,27734405.0,front,BOS,0.671,Boston Celtics,4
4,250,al jefferson,C,33,36,1,13.4,3.1,5.8,0.534,...,0.6,1.8,7.0,2018,9769821.0,front,IND,0.585,Indiana Pacers,11


In [93]:
df = ori_data
df.rename(columns=lambda x:x.replace('team_id','pos_num'), inplace=True)  
#set(df.iloc[:,2])
 
for i in range(len(df)):
    if (df.iloc[i,2] == 'PG' or df.iloc[i,2] =='SG' or df.iloc[i,2] =='PG-SG'): df.iloc[i,32] = 0; # backward = 0
    else: df.iloc[i,32] = 1    # forward = 1. 'SF-SG': corey brewer & shabazz muhammad(gone to CBA) temporarary: SF (index: 62, 375)

df.head()

,player_rank,player,pos,age,games,games_started,min_per_game,field_goals,field_goals_attempts,field_goals_pct,...,turnovers,personal_fouls,points_per_game,year,salary,new_pos,pos_num,team_points,team,team_rank
0,65,aaron brooks,PG,33,32,1,5.9,0.9,2.2,0.406,...,0.3,0.9,2.3,2018,2116955.0,back,0,0.573,Minnesota Timberwolves,13
1,179,aaron gordon,PF,22,58,57,32.9,6.5,14.9,0.434,...,1.8,1.9,17.6,2018,5504420.0,front,1,0.305,Orlando Magic,26
2,363,abdel nader,SF,24,48,1,10.9,1.0,3.1,0.336,...,0.7,0.9,3.0,2018,1167333.0,front,1,0.671,Boston Celtics,4
3,227,al horford,C,31,72,72,31.6,5.1,10.5,0.489,...,1.8,1.9,12.9,2018,27734405.0,front,1,0.671,Boston Celtics,4
4,250,al jefferson,C,33,36,1,13.4,3.1,5.8,0.534,...,0.6,1.8,7.0,2018,9769821.0,front,1,0.585,Indiana Pacers,11


In [94]:
factors = pd.concat([df.iloc[:,1],df.iloc[:,0],df.iloc[:,3:28],df.iloc[:,32:33],df.iloc[:,35]],axis=1)
y = df.iloc[:,30]

row = len(factors)
col = len(factors.columns)

factors.head()

,player,player_rank,age,games,games_started,min_per_game,field_goals,field_goals_attempts,field_goals_pct,3points,...,orb,drb,trb,assists,steals,blocks,turnovers,personal_fouls,pos_num,team_rank
0,aaron brooks,65,33,32,1,5.9,0.9,2.2,0.406,0.3,...,0.2,0.3,0.5,0.6,0.2,0.0,0.3,0.9,0,13
1,aaron gordon,179,22,58,57,32.9,6.5,14.9,0.434,2.0,...,1.5,6.4,7.9,2.3,1.0,0.8,1.8,1.9,1,26
2,abdel nader,363,24,48,1,10.9,1.0,3.1,0.336,0.5,...,0.3,1.2,1.5,0.5,0.3,0.2,0.7,0.9,1,4
3,al horford,227,31,72,72,31.6,5.1,10.5,0.489,1.3,...,1.4,5.9,7.4,4.7,0.6,1.1,1.8,1.9,1,4
4,al jefferson,250,33,36,1,13.4,3.1,5.8,0.534,0.0,...,0.8,3.1,4.0,0.8,0.4,0.6,0.6,1.8,1,11


In [120]:
factor_index = list(factors.columns[2:])
stat_value = corr_value = rsquare_value = pd.Series(factor_index)


for index in factor_index:
    corr_single_list = []
    t_single_list = []
    rsquare_single_list = []
    for s in range(2, col):
        xy = pd.concat([y, factors.iloc[:,s]], axis=1).dropna()
        #print(xy)
        # corr between single x and y
        corr = xy.corr().iloc[0][1]
        #print(corr)
        y = xy.iloc[:,0]
        x = xy.iloc[:,1]        
        beta = sum((x-np.mean(x))*(y-np.mean(y)))/sum((x-np.mean(x))**2) 
        const = np.mean(y)-beta*np.mean(x)          
        # variance  
        var=sum((y-const-beta*x)**2)/(len(y)-2)
        # std. var  
        sigma=np.sqrt(var) 
        # r_square
        rsquare = r2_score(y, (const+beta*x))
        #t-value  
        t=beta*np.sqrt(sum((x-np.mean(x))**2))/sigma
        
        t_single_list.append(t)
        corr_single_list.append(corr)
        rsquare_single_list.append(rsquare)
    
    t_single_se = pd.Series(t_single_list)    
    stat_value = pd.concat([stat_value,t_single_se],axis = 1)
    
    corr_single_se = pd.Series(corr_single_list)
    corr_value = pd.concat([corr_value,corr_single_se],axis = 1) 
    
    rsquare_single_se = pd.Series(rsquare_single_list)
    rsquare_value = pd.concat([rsquare_value,rsquare_single_se],axis = 1) 
    #print('----------------------------------------')

stat_value = pd.concat([stat_value,corr_value, rsquare_value],axis = 1)

In [121]:
stat_value

,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,age,7.319387,7.319387,7.319387,7.319387,7.319387,7.319387,7.319387,7.319387,7.319387,...,0.116319,0.116319,0.116319,0.116319,0.116319,0.116319,0.116319,0.116319,0.116319,0.116319
1,games,4.627815,4.627815,4.627815,4.627815,4.627815,4.627815,4.627815,4.627815,4.627815,...,0.049990,0.049990,0.049990,0.049990,0.049990,0.049990,0.049990,0.049990,0.049990,0.049990
2,games_started,14.220546,14.220546,14.220546,14.220546,14.220546,14.220546,14.220546,14.220546,14.220546,...,0.331937,0.331937,0.331937,0.331937,0.331937,0.331937,0.331937,0.331937,0.331937,0.331937
3,min_per_game,14.703472,14.703472,14.703472,14.703472,14.703472,14.703472,14.703472,14.703472,14.703472,...,0.346911,0.346911,0.346911,0.346911,0.346911,0.346911,0.346911,0.346911,0.346911,0.346911
4,field_goals,16.198156,16.198156,16.198156,16.198156,16.198156,16.198156,16.198156,16.198156,16.198156,...,0.391975,0.391975,0.391975,0.391975,0.391975,0.391975,0.391975,0.391975,0.391975,0.391975
5,field_goals_attempts,15.463003,15.463003,15.463003,15.463003,15.463003,15.463003,15.463003,15.463003,15.463003,...,0.370071,0.370071,0.370071,0.370071,0.370071,0.370071,0.370071,0.370071,0.370071,0.370071
6,field_goals_pct,3.321346,3.321346,3.321346,3.321346,3.321346,3.321346,3.321346,3.321346,3.321346,...,0.026389,0.026389,0.026389,0.026389,0.026389,0.026389,0.026389,0.026389,0.026389,0.026389
7,3points,9.257242,9.257242,9.257242,9.257242,9.257242,9.257242,9.257242,9.257242,9.257242,...,0.173934,0.173934,0.173934,0.173934,0.173934,0.173934,0.173934,0.173934,0.173934,0.173934
8,3points_attempts,9.390416,9.390416,9.390416,9.390416,9.390416,9.390416,9.390416,9.390416,9.390416,...,0.178077,0.178077,0.178077,0.178077,0.178077,0.178077,0.178077,0.178077,0.178077,0.178077
9,3points_pct,2.092795,2.092795,2.092795,2.092795,2.092795,2.092795,2.092795,2.092795,2.092795,...,0.010647,0.010647,0.010647,0.010647,0.010647,0.010647,0.010647,0.010647,0.010647,0.010647


In [110]:
xy = pd.concat([y, factors.iloc[:,1]], axis=1)#.dropna()
#print(xy)
        # corr between single x and y
xy.corr().iloc[0][1]

IndexError: index out of bounds